In [1]:
import utils, data, simulation
import os
import os.path as op
import pandas as pd
import numpy as  np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
checkAxon = True
drawAllDist = False

In [3]:
#create an instance of data processing class
data = data.processing()

In [4]:
#folder containing scripts
scriptpath = os.environ['SCRIPT_DATA']
outpath = op.join(scriptpath, 'data')
filename = 'stimulation_thresholds.csv'
df = pd.read_csv(op.join(outpath,'experiment_notes',filename))
df.columns = df.columns.str.strip()

df= df.rename(columns= {'Patient No':'Subject'})


In [5]:
op.join(outpath,'experiment_notes',filename)

'/mnt/viscog/FineLab/ArgusScripts/data/experiment_notes/stimulation_thresholds.csv'

In [6]:
# Remove the ones not used in the double electrode experiment
#df = df[df['Selected']!=0]

#change column name
df= df.rename(columns= {'Patient No':'Subject'})
# Average multiple measurements for a given session
avg_data = (df
.groupby(['Subject','Session', 'Electrode'])
.mean())

In [7]:
num_electrode = (df
.groupby(['Subject'])
.nunique())

In [8]:
num_electrode

,Session,Electrode,Threshold,Selected
Subject,,,,
12-005,2,12,10,2
12-104,2,14,11,2
13-101,2,12,13,2


In [9]:
# individual electrodes
avg_data= avg_data.reset_index()
avg_data.sort_values(by = ['Threshold'])


,Subject,Session,Electrode,Threshold,Selected
30,13-101,2019/05/31,B10,89.0,1.0
41,13-101,2019/10/7,B10,93.0,1.0
1,12-005,2019/05/29,A8,153.0,1.0
42,13-101,2019/10/7,B9,177.0,1.0
37,13-101,2019/05/31,F10,177.0,1.0
0,12-005,2019/05/29,A4,194.0,1.0
43,13-101,2019/10/7,C6,202.0,0.0
34,13-101,2019/05/31,C9,202.0,0.0
45,13-101,2019/10/7,F10,202.0,1.0
32,13-101,2019/05/31,C6,210.0,1.0


In [10]:
medians = (avg_data
.groupby(['Subject'])
.median())
medians.index

Index(['12-005', '12-104', '13-101'], dtype='object', name='Subject')

In [11]:
# due to differences in the way matlab and python
# computes percentiles we use the following function
# from https://stackoverflow.com/questions/58424704/output-produced-by-python-numpy-percentile-not-same-as-matlab-prctile
def matlab_percentile(x, p):
    p = np.asarray(p, dtype=float)
    n = len(x)
    p = (p-50)*n/(n-1) + 50
    p = np.clip(p, 0, 100)
    return np.percentile(x, p)

In [12]:
t_table = pd.DataFrame()
for subject in medians.index:
    query_string = 'Subject==' + '"' + subject + '"'
    print(query_string)  
    # select a single subject
    selected = avg_data.query(query_string)

    #calculate interquartile range of values in the 'points' column
    median = selected['Threshold'].median()
    #q75, q25 = np.percentile(selected['Threshold'], [75 ,25], interpolation='midpoint')
    q75 = np.round(matlab_percentile(selected['Threshold'],75),1)
    q25 = np.round(matlab_percentile(selected['Threshold'],25),1)

    iqr = q75 - q25
    t_table = t_table.append({'Subject': subject, 'Median': median, 'Q25-Q75': (q25, q75),
                              'IQR': iqr, 'Minimum': np.min(selected['Threshold']), 
                             'Maximum': np.max(selected['Threshold'])}, ignore_index=True)

Subject=="12-005"
Subject=="12-104"
Subject=="13-101"


In [13]:
t_table

,IQR,Maximum,Median,Minimum,Q25-Q75,Subject
0,224.0,516.0,274.0,153.0,"(244.0, 468.0)",12-005
1,181.2,677.0,613.0,217.0,"(463.8, 645.0)",12-104
2,70.2,403.0,218.0,89.0,"(195.8, 266.0)",13-101


## Catch statistics

In [14]:
filename = 'S_12-005_12-104_13-101__collated_both_sessions.csv'

(double_x2, catch, subjectdata) = data.get_two_point(outpath, scriptpath, filename)

catch_means = pd.DataFrame()
for subject in medians.index:
    query_string = 'subject_id==' + '"' + subject + '"'
    print(query_string)  
    #select subject
    split = catch.query(query_string)
    # total number of trials per participant
    n= len(split)
    # times they perceived nothing when no stim
    hit = len(split[split['pts_number_processed']==0])
    # times they perceived >0 shapes
    fa = len(split[split['pts_number_processed']>0])
    print(n)
    print(hit)
    print(fa)
    catch_means = catch_means.append( {'subject_id': subject, 'false alarm rate': fa,
                             'total no of trials' : n, 'fa/n' : fa/n }, 
                             ignore_index= True)


subject_id=="12-005"
15
15
0
subject_id=="12-104"
12
10
2
subject_id=="13-101"
16
16
0


In [15]:
catch_means

,fa/n,false alarm rate,subject_id,total no of trials
0,0.000000,0.0,12-005,15.0
1,0.166667,2.0,12-104,12.0
2,0.000000,0.0,13-101,16.0
